# PARTE 1: Extração de Dados

## Configurando Ambiente

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Exibir versão do Python
import platform
platform.python_version()

'3.9.9'

In [2]:
try: # Checando se Scrapy está instalado
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

     |████████████████████████████████| 254 kB 7.9 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 1.5 MB/s  eta 0:00:01
     |████████████████████████████████| 55 kB 1.4 MB/s  eta 0:00:01
     |████████████████████████████████| 47 kB 2.1 MB/s  eta 0:00:01
     |████████████████████████████████| 3.2 MB 6.3 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 19.3 MB/s eta 0:00:01
     |████████████████████████████████| 255 kB 66.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 23.9 MB/s eta 0:00:01     |██████████████████████████▌     | 5.7 MB 23.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 31.4 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 3.0 MB/s  eta 0:00:01
     |████████████████████████████████| 155 kB 32.7 MB/s eta 0:00:01
     |████████████████████████████████| 74 kB 1.4 MB/s  eta 0:00:01
  Using cached idna-3.3-py3-none-any.whl (61 kB)
Using legacy 'setup.py install' for protego, since package 'wh

In [3]:
class QuotesSpider(scrapy.Spider):
    name = "quotes"
    # URLs
    start_urls = [
        'http://quotes.toscrape.com/page/1/'
    ]
    
    # Parse da página principal a ser crawleada
    def parse(self, response):
       
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract(),
                'author': quote.css('span small::text').extract(),
                'tags': quote.css('div.tags a.tag::text').extract()
            }

In [4]:
process = CrawlerProcess(get_project_settings())

# Iniciando processo
process.crawl(QuotesSpider)
process.start()

2021-12-11 16:01:03 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-12-11 16:01:03 [scrapy.utils.log] INFO: Versions: lxml 4.6.4.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.9.9 (main, Nov 20 2021, 21:30:06) - [GCC 11.1.0], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 36.0.0, Platform Linux-5.15.7-arch1-1-x86_64-with-glibc2.33
2021-12-11 16:01:03 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-12-11 16:01:03 [scrapy.crawler] INFO: Overridden settings:
{}
2021-12-11 16:01:03 [scrapy.extensions.telnet] INFO: Telnet Password: 5bdb719deae0a6e6
2021-12-11 16:01:03 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2021-12-11 16:01:03 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermi

2021-12-11 16:01:04 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://quotes.toscrape.com/page/1/> (referer: None)
2021-12-11 16:01:04 [scrapy.core.scraper] DEBUG: Scraped from <200 http://quotes.toscrape.com/page/1/>
{'text': ['“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”'], 'author': ['Albert Einstein'], 'tags': ['change', 'deep-thoughts', 'thinking', 'world']}
2021-12-11 16:01:04 [scrapy.core.scraper] DEBUG: Scraped from <200 http://quotes.toscrape.com/page/1/>
{'text': ['“It is our choices, Harry, that show what we truly are, far more than our abilities.”'], 'author': ['J.K. Rowling'], 'tags': ['abilities', 'choices']}
2021-12-11 16:01:04 [scrapy.core.scraper] DEBUG: Scraped from <200 http://quotes.toscrape.com/page/1/>
{'text': ['“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”'], 'author': ['Albert Einstein'], 'tags': ['insp

# PARTE 2: Gerando Arquivo de Saída

## Ao executar esse trecho, reinicie o jupyter notebook

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# Exibir versão do Python
import platform
platform.python_version()

try: # Checando se Scrapy está instalado
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

In [2]:
import json

class JsonWriterPipeline(object):

    # Função para gerar/abrir arquivo JSON
    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    # Fechar arquivo após escrita
    def close_spider(self, spider):
        self.file.close()

    # Inserir itens coletados da página WEB no arquivo JSON criado
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    # URLs
    start_urls = [
        'http://quotes.toscrape.com/page/1/',
        'http://quotes.toscrape.com/page/2/',
    ]
    
    # Configuração obrigatória de pipeline para geração de arquivo de saída
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, 
        'FEED_FORMAT':'json',                                 
        'FEED_URI': 'quoteresult.json'                        
    }
    
    # Parse da página principal a ser crawleada
    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').extract()[0],
                'author': quote.css('span small::text').extract()[0],
                'tags': quote.css('div.tags a.tag::text').extract()
            }

In [4]:
process = CrawlerProcess(get_project_settings())

# Iniciando processo
process.crawl(QuotesSpider)
process.start()

2021-12-11 16:01:46 [scrapy.utils.log] INFO: Scrapy 2.5.1 started (bot: scrapybot)
2021-12-11 16:01:46 [scrapy.utils.log] INFO: Versions: lxml 4.6.4.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.9.9 (main, Nov 20 2021, 21:30:06) - [GCC 11.1.0], pyOpenSSL 21.0.0 (OpenSSL 1.1.1l  24 Aug 2021), cryptography 36.0.0, Platform Linux-5.15.7-arch1-1-x86_64-with-glibc2.33
2021-12-11 16:01:46 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-12-11 16:01:46 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30}
2021-12-11 16:01:46 [py.warnings] WARNING: /home/gabrielfv/Projects/UFC/ck0223-mineracao/venv/lib/python3.9/site-packages/scrapy/extensions/feedexport.py:247: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)



In [ ]:
import pandas as pd
# Carregando JSON criado para visualizar saída
output = pd.read_json('quoteresult.jl', lines=True)
output

## ---------------------------------------------------------------------------------------------------------------------------------

## Outro Exemplo: site SEFAZ

In [ ]:
try: # Checando se Scrapy está instalado
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

In [ ]:
import json

class JsonWriterPipeline(object):

    # Função para gerar/abrir arquivo JSON
    def open_spider(self, spider):
        self.file = open('normaresult.jl', 'w')

    # Fechar arquivo após escrita
    def close_spider(self, spider):
        self.file.close()

    # Inserir itens coletados da página WEB no arquivo JSON criado
    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [ ]:
class SefazItem(scrapy.Item):
    decreto = scrapy.Field()
    publicacao_dou = scrapy.Field()
    ementa = scrapy.Field()
    norma = scrapy.Field()

In [ ]:
def get_publicacao_dou(response):
    publicacao_dou = response.css('div.tituloPublicacao ::text')[0].extract().strip()
        
    return publicacao_dou


def get_ementa(response):
    # retrieve the ementa <div>
    ementa_div = response.css('p.ementa::text').extract()

    ementa_list = []
    for i in ementa_div:
        ementa_list.append(i.strip())

    # that ementa_list is broken into some sentences; the function below generates the joint text
    ementa = ' '.join(ementa_list)
    return ementa

def get_norma_elements(response):
    # retrieve the norma <div>
    norma = response.css('div.divSegmentos')

    # declaring the list
    norma_list = []
    for i in norma:
        # retrieve only the text <p> inside the <div>
        tmp = i.css('span::text')[0].extract().strip()
        # add each element into the norma list
        norma_list.append(tmp)

    return norma_list


In [ ]:
import logging

class NormaSpider(scrapy.Spider):
    name = 'norma'
    start_urls = [
        'http://normas.receita.fazenda.gov.br/sijut2consulta/link.action?visao=anotado&idAto=95072'
    ]

    # Configuração obrigatória de pipeline para geração de arquivo de saída
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, 
        'FEED_FORMAT':'json',                                 
        'FEED_URI': 'normaresult.json'                        
    }
    
    def parse(self, response):
        # Output file into json of ONE single page
        items = SefazItem()
        items['publicacao_dou'] = get_publicacao_dou(response)
        items['ementa'] = get_ementa(response)
        items['norma'] = get_norma_elements(response)

        yield items


In [ ]:
process = CrawlerProcess(get_project_settings())

# Iniciando processo
process.crawl(NormaSpider)
process.start()

In [ ]:
import pandas as pd
# Carregando JSON criado para visualizar saída
output = pd.read_json('normaresult.jl', lines=True)
output

In [ ]:
 pd.options.display.max_colwidth = 5000

In [ ]:
output

## Links Uteis